# Análisis y traducción de tweets de texto común a ASCII

Notebook generadora del dataset para utilizar en el análisis. Toma la base de datos, remueve las palabras y signos a ignorar, traduce los emojis a ASCII y guarda el dataset final.

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from TweetAnalysis import hermes

#### Cargo la base de datos clasificada y genero las variables de importancia

In [3]:
TW = pd.read_csv('base_corr.csv')

Estos son los tweets no clasificados

In [4]:
DATA = pd.read_csv('tweets.csv',sep=';')

MSJ contiene al tweet, Tag si es meteorológico o no, Num el id del tweet, Who el nombre de quien los clasificó y User el id de quien twiteó (eso lo tuvimos que sacar de la base de datos original).

In [5]:
MSJ = TW['Twit']
TAG = TW['Tag']
NUM = TW['Num']
WHO = TW['Nombre']
USER = DATA['user_id'][np.asarray(NUM)]

In [6]:
FINAL = pd.DataFrame(list(zip(MSJ, TAG, NUM, USER, WHO)), columns = ['MSJ', 'TAG', 'NUN', 'USER', 'WHO'])

## La función Hermes: normaliza los tweets y borra el spam meteorológico

In [9]:
FINAL = hermes(FINAL, 'MSJ', 'USER', 5)
FINAL.to_csv(r'BaseOrt.csv', sep = ';')

## Corrector ortográfico

A mano :( Se puede hacer con Hunspell, si lo lográs instalar, suerte!

In [10]:
ORT = pd.read_csv('BaseOrt2.csv', sep = ';')

In [11]:
ORT

,esta lloviendo
0,quiero ir centro mañana pero seguro llueve
1,si llueve mucho \U0001f612
2,si llueve hago \U0001f917
3,hasta cuando voy seguir teniendo miedo torment...
4,sea estacion sea no hay plata ahora llueve...
5,llueve no tengo internet espera tremenda tarde
6,que linda noche lluvia \U0001f496
7,miercoles mucho calor previo lluvias
8,me gusta cuando llueve porque parece estan fri...
9,noviembre sin ti sentir lluvia dice llorando t...
